Esta notebook esta destinada a analizar las respuestas de la api, entender los datos que vienen en los registros, filtrar los utiles y eliminar los que no necesitamos.

Tuvimos un inconveniente a la hora de realizar esto que planteamos, y es que, al intentar cargar los resultados en memoria, desbordaba la RAM de la notebook.

Por este motivo, fue necesario conectar el colab a una instancia especifica de Google, para esto, tuvimos que levantar una instancia en Google Cloud Platform, puntualmente una n1-highmem-4 ubicada en la zona de Google us-west1-b. Esta instancia tiene 4 cores, y 30gb de RAM. Recursos que fueron suficientes para poder trabajar con los resultados de la notebook de Querys.

Con la desventaja que este .ipynb debe ser ejecutado con la cuenta donde está levantada la instancia en cuestión.

Como no nos encontramos en la instancia tradicional que corre la notebook, es necesaio que nos autentiquemos de forma diferente para usar Google Cloud Storage. 

Asi que, nos autenticamos de la siguiente manera:

In [ ]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=aeyEUJpfJaPTnV3vi6QS1IR5YBTf61&prompt=consent&access_type=offline&code_challenge=mP6uSZe94akiVb-bu3AKbjM838PBVtXhMG6a8yFnkv0&code_challenge_method=S256

Enter verification code: 4/1AX4XfWhpPWfCmWVOXM1a76f5-NBBunhtoAdSoxV9MLOgaDmNaS1cCBBw1n8

You are now logged in as [an2021dic15@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


Vamos a descargar el archivo de results del bucket, subido en la notebook anterior. De la misma forma que antes, declaramos dos variables, una con el nombre del proyecto y la otra con el nombre del bucket, y nos autenticamos.

Y luego, copiamos del bucket al filesystem, el archivo results.pkl

In [83]:
project_id = 'cryptic-opus-335323'
bucket_name = 'bdm-unlu'
!gsutil cp gs://{bucket_name}/querys/results.pkl results.pkl 

Copying gs://bdm-unlu/querys/results.pkl...
- [1 files][  1.4 GiB/  1.4 GiB]   38.2 MiB/s                                   
Operation completed over 1 objects/1.4 GiB.                                      


Como cada uno de los workers de la notebook anterior, hizo un append al archivo, no basta con hacer un unico load, para obtener los resultados, dado que cada worker hizo un dump, cada vez que guardó resultados de una petición.

Por este motivo es que hacemos load del archivo, hasta que tire una exception, que será cuando el archivo ya no tenga mas nada para leer, lo que resulta en la exception "Ran out of input".

In [84]:
import pickle
open_file_read = open("results.pkl", "rb")
raw_results = []
has_load = True
while has_load:    
  try:
    raw_results.extend(pickle.load(open_file_read))
  except Exception as e:
    print(e)
    has_load = False

Ran out of input


Corroboramos la cantidad de registros cargados en memoria

In [85]:
len(raw_results)

308017

Sabemos que la respuesta de la API, tiene una clave llamada "attributes", donde se encuentran caracteristicas del inmueble.

Y a su vez, cada atributo, tiene diferentes claves, ID, NAME, etc. que corresponden a ese atributo en particular.

Esta función dado un unico registro y una clave, obtiene de esa respuesta, de todos los atributos, la clave indicada por parametro.

In [86]:
def get_atributes(result, key):
  atributes_names = []
  for atribute in result["attributes"]:
    atributes_names.append(atribute[key])
  return atributes_names

La funcion a continuación, dada una lista de resultados y una clave, obtiene la lista de atributos univoca, identificandolos por la clave dada.

In [87]:
def get_unique_atribute_list(results, key = "name"):
  atribute_list = []
  for result in results:
    atributes = get_atributes(result, key)
    for atribute in atributes:
      if atribute not in atribute_list:
        atribute_list.append(atribute)
  return atribute_list

Eso nos permite conocer, de los 308017 registros todos los posibles atributos que tienen los registros.

In [88]:
get_unique_atribute_list(raw_results)

['Condición del ítem',
 'Aire acondicionado',
 'Línea telefónica',
 'Dormitorios',
 'Superficie cubierta',
 'Baños',
 'Ambientes',
 'Superficie total',
 'Operación',
 'Inmueble',
 'Tour virtual',
 'Logo',
 'Cantidad máxima de baños',
 'Cantidad máxima de dormitorios',
 'Cantidad máxima de metros cubiertos',
 'Cantidad máxima de ambientes',
 'Cantidad máxima de metros totales',
 'Cantidad mínima de baños',
 'Cantidad mínima de dormitorios',
 'Cantidad mínima de metros cubiertos',
 'Cantidad mínima de ambientes',
 'Cantidad mínima de metros totales',
 'Nombre del emprendimiento',
 'Fecha de entrega',
 'Número de oficinas']

Vemos que hay bastantes que son intuitivos, pero hay otros que ;laman la atencion. Como por ejemplo los de "Cantidad máxima de baños", "Cantidad máxima de dormitorios", ... 

Analizando encontramos que se trata de Emprendimientos, publicados en MercadoLibre, tales como [este](https://departamento.mercadolibre.com.ar/MLA-913664246-edificio-en-piedras-al-1300-san-telmo-_JM).

Y encontramos que todos estos emprendimientos, tienen un atributo, cuyo ID es DEVELOPMENT_NAME.

Para tratarlos, hicimos una funcion que determina si una respuesta determinada, contiene o no, en su lista de atributos, un valor especifico, en una clave determinada.

In [89]:
def contains(result, key, value):
  for x in result:
    if x[key] == value:
      return True
  return False

La utilizamos para ver, de todos los registros, cuales contenían en su lista de atributos, un atributo cuyo "id", sea DEVELOPMENT_NAME.

Y guardamos en una lista llamada in_development, los inmuebles que eran emprendimientos, y los que no, en una lista llamada in_use.

In [90]:
in_development = []
in_use = []

atribute_list = []
for raw_result in raw_results:
  if contains(raw_result["attributes"], "id", "DEVELOPMENT_NAME"):
    in_development.append(raw_result)
  else:
    in_use.append(raw_result)

De esta manera, si vemos la longitud de la lista in_development, podremos ver, la cantidad de inmuebles que son de tipo emprendimiento, publicados en MercadoLibre.

In [91]:
len(in_development)

1640

Si ahora vemos la lista univoca de atributos, de los inmuebles que quedaron en la lista in_use, vemos que ya no aparecen los atributos que nos llamaban la atención, tales como "Cantidad mínima de metros cubiertos", "Cantidad mínima de ambientes", etc.

In [92]:
get_unique_atribute_list(in_use)

['Condición del ítem',
 'Aire acondicionado',
 'Línea telefónica',
 'Dormitorios',
 'Superficie cubierta',
 'Baños',
 'Ambientes',
 'Superficie total',
 'Operación',
 'Inmueble',
 'Tour virtual',
 'Número de oficinas']

Un valor que llama medianamente la atención es 'Tour virtual', por lo que verificamos si se trata de un valor atipico, o si la mayoría de los registros lo tienen.

In [93]:
records_with_virtual_tour = 0
for value in in_use:
  if contains(value["attributes"], "id", "WITH_VIRTUAL_TOUR"):
    records_with_virtual_tour += 1
records_with_virtual_tour

245235

Vemos que dicha cantidad tienen este atributo, y analizando, vimos que se trata de un video, mostrando el interior del departamento y la entrada del edificio.

Por ultimo, el atributo de "Número de oficinas", llama la atención, por lo que también hicimos un analisis, para ver de que se trata.

In [94]:
records_with_office_number = []
for value in in_use:
  if contains(value["attributes"], "name", "Número de oficinas"):
    records_with_office_number.append(value)
len(records_with_office_number)

4

Vemos que solo esa cantidad de registros, tienen este atributo, por lo que es un atributo bastante propenso a ser borrado.

Obtenemos el permalink para poder ver en la web de que inmueble se trata

In [95]:
records_with_office_number[0]["permalink"]

'https://departamento.mercadolibre.com.ar/MLA-1108741528-hermoso-loft-con-techos-a-35-m-de-altura-_JM'

Y vemos que son propiedades utilizadas como oficina, por lo que los removemos de la lista de registros.

In [96]:
for record_with_office_number in records_with_office_number:
  in_use.remove(record_with_office_number)

Y si ahora corroboramos la lista de atributos, vemos que esta limpia, con todos atributos que nos parecen válidos para describir un inmueble, de la forma que queremos, para el analisis que estamos buscando realizar.

In [97]:
get_unique_atribute_list(in_use)

['Condición del ítem',
 'Aire acondicionado',
 'Línea telefónica',
 'Dormitorios',
 'Superficie cubierta',
 'Baños',
 'Ambientes',
 'Superficie total',
 'Operación',
 'Inmueble',
 'Tour virtual']

Hasta aca un analisis de los diferentes atributos que contenian los registros, y una primera aproximación al filtrado.



---



Ahora vamos a seleccionar los atributos mas importantes y armar un primer DataFrame.

Obtenemos la lista de atributos univocos, pero ahora utilizando el id como identificador, y no el name. Y la guardamos en una variable llamada property_attributes.

In [98]:
property_attributes = get_unique_atribute_list(in_use, "id")

Esta funcion, dado un registro y una lista de claves, encuentra en la respuesta de la API el valor buscado, o devuelve None.

In [99]:
def fetch_nested_value(record, keys):  
  try:
    for key in keys: 
      record = record.get(key) 
    return record
  except: 
    return None

Este bloque de codigo, dado un registro y un valor de ID determinado,, encuentra el atributo cuyo id es igual al pasado por parametro.

In [100]:
def fetch_attribute_by_id(record, attr_id): 
  try:
    for attr in record["attributes"]: 
      if (attr["id"] == attr_id):
        return attr["value_name"]
  except:
    return None    

Esta es la funcion que mapea de un registro de la API, con campos que nos interesan y campos que no, a un diccionario de python, unicamente con campos que son utiles.

Siempre contiene los atributos presentes en property_attributes, en caso de no estar en la respuesta de la API, los asigna con valor None.

In [101]:
def process_record(record, expected_attributes): 
  result = {}    
  result["title"] = record["title"]
  result["permalink"] = record["permalink"]

  result["city"] = fetch_nested_value(record, ["location","city","name"])
  result["state"] = fetch_nested_value(record, ["location","state","name"])
  result["latitude"] = fetch_nested_value(record, ["location","latitude"])
  result["longitude"] = fetch_nested_value(record, ["location","longitude"])
  result["neighborhood"] = fetch_nested_value(record, ["location","neighborhood","name"])
  
  result["seller_id"] = fetch_nested_value(record, ["seller","id"])
  result["seller_city"] = fetch_nested_value(record, ["seller_address","city","name"])
  result["seller_state"] = fetch_nested_value(record, ["seller_address","state","name"])
  result["real_estate_agency"] = fetch_nested_value(record, ["seller","real_estate_agency"])
  result["seller_cancelations"] = fetch_nested_value(record, ["seller","seller_reputation","metrics","cancellations","value"])
  result["seller_claims"] = fetch_nested_value(record, ["seller","seller_reputation","metrics","claims","value"])
  result["seller_handling_time"] = fetch_nested_value(record, ["seller","seller_reputation","metrics","delayed_handling_time","value"])
  result["seller_sales"] = fetch_nested_value(record, ["seller","seller_reputation","metrics","sales","completed"])
  
  result["currency_id"] = record["currency_id"]
  result["price"] = record["price"]

  for attr in expected_attributes:
    result[attr.lower()] = fetch_attribute_by_id(record, attr)
  
  return result

Procesa todos los registros, y los guarda en una lista (records)

In [102]:
records = []
for record in in_use:
  try:
    records.append(process_record(record, property_attributes))
  except Exception as e:    
    print(e)

Importamos la libreria de pandas, y creamos un dataframe a partir de la lista de diccionarios.

In [103]:
import pandas as pd
pd.set_option('display.max_colwidth', 150)
data = pd.DataFrame.from_dict(records)

Imprimimos el resultado de hacerle un head al dataframe mencionado.

In [104]:
data.head()

,title,permalink,city,state,latitude,longitude,neighborhood,seller_id,seller_city,seller_state,real_estate_agency,seller_cancelations,seller_claims,seller_handling_time,seller_sales,currency_id,price,item_condition,has_air_conditioning,has_telephone_line,bedrooms,covered_area,full_bathrooms,rooms,total_area,operation,property_type,with_virtual_tour
0,Departamento Venta 3 Ambientes Palermo Amenities Cochera,https://departamento.mercadolibre.com.ar/MLA-1116767893-departamento-venta-3-ambientes-palermo-amenities-cochera-_JM,Capital Federal,Capital Federal,-34.579563,-58.433160,Palermo Hollywood,314431007,Palermo,Capital Federal,True,0.0,0.0,0.0,0.0,USD,270000.0,Usado,No,No,2,85 m²,3,3,85 m²,Venta,Departamento,No
1,Venta Departamento 3 Ambientes Piso 15 Palermo,https://departamento.mercadolibre.com.ar/MLA-1107347883-venta-departamento-3-ambientes-piso-15-palermo-_JM,Capital Federal,Capital Federal,-34.573490,-58.436577,Palermo Hollywood,466036843,Monserrat,Capital Federal,True,0.0,0.0,0.0,0.0,USD,148500.0,Usado,None,None,2,46 m²,1,3,55 m²,Venta,Departamento,No
2,Venta Departamento 3amb Con Cochera Triplex Palermohollywood,https://departamento.mercadolibre.com.ar/MLA-1113461025-venta-departamento-3amb-con-cochera-triplex-palermohollywood-_JM,Capital Federal,Capital Federal,-34.585620,-58.440120,Palermo Hollywood,419112633,Palermo Chico,Capital Federal,True,0.0,0.0,0.0,0.0,USD,215000.0,Usado,No,Sí,2,93 m²,3,3,102 m²,Venta,Departamento,None
3,Departamento - Palermo Hollywood - Venta Monoambiente Quartier Dorrego,https://departamento.mercadolibre.com.ar/MLA-921699674-departamento-palermo-hollywood-venta-monoambiente-quartier-dorrego-_JM,Capital Federal,Capital Federal,-34.577396,-58.439194,Palermo Hollywood,694286242,Belgrano,Capital Federal,True,0.0,0.0,0.0,0.0,USD,180000.0,Nuevo,Sí,No,0,40 m²,1,1,40 m²,Venta,Departamento,No
4,Departamento - Palermo Hollywood,https://departamento.mercadolibre.com.ar/MLA-1116714973-departamento-palermo-hollywood-_JM,Capital Federal,Capital Federal,-34.579685,-58.429800,Palermo Hollywood,418539414,Palermo Chico,Capital Federal,True,0.0,0.0,0.0,0.0,USD,135000.0,Nuevo,No,No,1,46 m²,1,2,52 m²,Venta,Departamento,No


Guardamos el dataset en un archivo llamado dataset.csv, y lo guardamos en el bucket, en el path bdm-unlu/attributes/dataset.csv

In [106]:
data.to_csv("dataset.csv")

!gsutil cp dataset.csv gs://{bucket_name}/attributes/dataset.csv

Copying file://dataset.csv [Content-Type=text/csv]...
- [1 files][ 97.5 MiB/ 97.5 MiB]                                                
Operation completed over 1 objects/97.5 MiB.                                     


Algunos Atributos que vemos en la interfaz web, accediendo por el permalink, que no estan en las respuestas de la API, son:

* Cocheras
* Disposición
* Antigüedad
* Cantidad de pisos
* Expensas
* Descripcion

Para obtener estos, hicimos un [html scrapper](https://colab.research.google.com/drive/1bZE1LxYHKdU_7sd7aTB_6EtfpxXNZ_ZG).


En otra notebook, comenzamos a hacer un primer Analisis Exploratorio, para ver graficamente los datos que teníamos hasta el momento. [EDA Notebook](https://colab.research.google.com/drive/1cB-0nSc58VGIT8FRBt4llbzxHoFKVKZp#gceVm=cryptic-opus-335323/us-west1-b/colab-1-vm&scrollTo=NtmM77ofAZuW).